In [1]:
from pytorch_tabnet.tab_model import TabNetRegressor

import torch
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error

import pandas as pd
import numpy as np
np.random.seed(0)

import os

from torch import nn
import torch
from torch import tensor 
from torch.autograd import Variable
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
from sklearn.model_selection import KFold
import torch
import itertools
from sklearn.metrics import roc_auc_score, accuracy_score
from tqdm.notebook import tqdm
from pathlib import Path


# Download dataset

it is preprocessed in excel
For week day 1~7: mon ~ sun

In [133]:
data = pd.read_csv('orders_autumn_2020 (1).csv')

In [334]:
data

,TIMESTAMP,Hour,Min,AM_PM,Hour_24,Weekday,ACTUAL_DELIVERY_MINUTES - ESTIMATED_DELIVERY_MINUTES,ITEM_COUNT,USER_LAT,USER_LONG,VENUE_LAT,VENUE_LONG,ESTIMATED_DELIVERY_MINUTES,ACTUAL_DELIVERY_MINUTES,CLOUD_COVERAGE,TEMPERATURE,WIND_SPEED,PRECIPITATION,kfold
0,8/1/2020,6,7,AM,6,6,-19,1,60.158,24.946,60.160,24.946,29,10,0.0,15.0,3.53644,0.0,4
1,8/1/2020,6,17,AM,6,6,-7,8,60.163,24.927,60.153,24.910,39,32,0.0,15.0,3.53644,0.0,3
2,8/1/2020,6,54,AM,6,6,-17,4,60.161,24.937,60.162,24.939,23,6,0.0,15.0,3.53644,0.0,4
3,8/1/2020,7,9,AM,7,6,-2,3,60.185,24.954,60.190,24.911,28,26,0.0,16.7,3.52267,0.0,5
4,8/1/2020,7,10,AM,7,6,-1,2,60.182,24.955,60.178,24.949,27,26,0.0,16.7,3.52267,0.0,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18424,9/30/2020,7,27,PM,19,3,-1,1,60.170,24.946,60.170,24.945,24,23,0.0,12.8,2.34419,0.0,0
18425,9/30/2020,7,36,PM,19,3,-8,6,60.185,24.944,60.178,24.949,23,15,0.0,12.8,2.34419,0.0,5
18426,9/30/2020,7,39,PM,19,3,-14,3,60.186,24.949,60.188,24.956,25,11,0.0,12.8,2.34419,0.0,4
18427,9/30/2020,7,42,PM,19,3,-11,3,60.172,24.929,60.168,24.930,21,10,0.0,12.8,2.34419,0.0,3


In [134]:
if "kfold" not in data.columns:
    print("split data")
    data["kfold"] = np.random.choice([0,1, 2, 3, 4, 5], p =[1/6,1/6,1/6,1/6,1/6,1/6] ,size=(data.shape[0],))


split data


# Define categorical features,  target

In [139]:
categorical_columns = ['PRECIPITATION','CLOUD_COVERAGE','ITEM_COUNT' , 'Weekday' ,'Hour_24', 'Min']
unused_feat = ['kfold', 'ESTIMATED_DELIVERY_MINUTES', 'ACTUAL_DELIVERY_MINUTES', 'AM_PM', 'TIMESTAMP', 'ACTUAL_DELIVERY_MINUTES - ESTIMATED_DELIVERY_MINUTES']
target = 'ACTUAL_DELIVERY_MINUTES'
features = [ col for col in data.columns if col not in unused_feat+[target]] 

cat_idxs = [ i for i, f in enumerate(features) if f in categorical_columns]


In [140]:
features

['Hour',
 'Min',
 'Hour_24',
 'Weekday',
 'ITEM_COUNT',
 'USER_LAT',
 'USER_LONG',
 'VENUE_LAT',
 'VENUE_LONG',
 'CLOUD_COVERAGE',
 'TEMPERATURE',
 'WIND_SPEED',
 'PRECIPITATION']

# drop na data

In [141]:
for feature in features:
    if np.any(np.isnan(data[feature].values)):
        print(feature)
    if np.all(np.isfinite(data[feature].values))== False:
        print(feature)

CLOUD_COVERAGE
CLOUD_COVERAGE
TEMPERATURE
TEMPERATURE
WIND_SPEED
WIND_SPEED


In [142]:
data=data.dropna().reset_index(drop=True)


# Load data and split

In [179]:
X_test = data[data.kfold == 5][features].values
y_test = data[data.kfold == 5][target].values.reshape(-1, 1)

# Network parameters

In [221]:
# Augmented
fold = 5
max_epochs = 200 
import torch
import itertools
from sklearn.metrics import confusion_matrix
from pytorch_tabnet.tab_model import TabNetClassifier
from sklearn.metrics import roc_auc_score, accuracy_score
from tqdm.notebook import tqdm


def find_bestpar(data):
    min_err= 9999
    # Store maximum hypterparameter set
    max_hy = []
    # define hyperparameter space :
    n_ = [4,8,16]                               
    lr_ = [0.02, 0.01] # learning rate
    w_ = [0.01, 0.001,]                 # weight decay
    g_ = [0.99, 0.9]                     # scheduler params - gamma
    ss_ = [10, 20]                         # scheduler params - step_size
    
    all_ = [n_, lr_, w_, g_, ss_]
    h_space = [s for s in itertools.product(*all_)]
    
    for hy in tqdm(h_space):
        """===================Cross Validation==================="""        
        test_score_res = []
        for i in range(fold):
            #print("fold ", i)
            clf = TabNetRegressor(n_a = hy[0],
                                n_d = hy[0],
                                optimizer_params = dict(lr=hy[1], weight_decay=hy[2]),
                                scheduler_params={"step_size":hy[4], "gamma":hy[3]},
                                scheduler_fn=torch.optim.lr_scheduler.StepLR,
                                verbose=0,
                                cat_idxs=cat_idxs )
            
            df_train = data[data.kfold != i]  
            df_valid = data[data.kfold == i]    
            X_train = df_train[features].values
            y_train = df_train[target].values.reshape(-1, 1)
            X_valid = df_valid[features].values
            y_valid = df_valid[target].values.reshape(-1, 1)

            clf.fit(
                    X_train=X_train, y_train=y_train,
                    eval_set=[(X_train, y_train), (X_valid, y_valid)],
                    eval_name=['train', 'valid'],
                    eval_metric=['rmsle', 'mae', 'rmse', 'mse'],
                    max_epochs=max_epochs,
                    patience=20,
                    batch_size=1024, virtual_batch_size=128,
                    num_workers=0,
                    drop_last=False
                ) 
            preds = clf.predict(X_test)
            y_true = y_test
            test_score = mean_squared_error(y_pred=preds, y_true=y_true)
            test_score_res.append(test_score)
            print('[%3d/%4d] '%(i+1, fold), 'Test score: %.3f'%test_score)
        print("Test score average: %3f"%np.mean(test_score_res))
        if np.mean(test_score_res) < min_err:
            #print("Find new maximum AUC!!")
            max_hy = hy
            min_err = np.mean(test_score_res)
    
    return max_hy

In [222]:

def bestpar_tuning(data, max_hy):
    hy = max_hy
    print("Max hy:" ,hy)
    """test data """
  
    # Orginal hyperparameter space 
 
    test_score_res = []
    clf_res =[] 
    fold = [0,1,2,3,4]
        
          
    for i in tqdm(fold):
        clf = TabNetRegressor(n_a = hy[0],
                                n_d = hy[0],
                                optimizer_params = dict(lr=hy[1], weight_decay=hy[2]),
                                scheduler_params={"step_size":hy[4], "gamma":hy[3]},
                                scheduler_fn=torch.optim.lr_scheduler.StepLR,
                                verbose=0,
                                cat_idxs=cat_idxs )
            
        df_train = data[data.kfold != i]  
        df_valid = data[data.kfold == i]  
            
        X_train = df_train[features].values
        y_train = df_train[target].values.reshape(-1, 1)

        X_valid = df_valid[features].values
        y_valid = df_valid[target].values.reshape(-1, 1)

        clf.fit(
                    X_train=X_train, y_train=y_train,
                    eval_set=[(X_train, y_train), (X_valid, y_valid)],
                    eval_name=['train', 'valid'],
                    eval_metric=['rmsle', 'mae', 'rmse', 'mse'],
                    max_epochs=max_epochs,
                    patience=50,
                    batch_size=1024, virtual_batch_size=128,
                    num_workers=0,
                    drop_last=False
                ) 
        preds = clf.predict(X_test)

        y_true = y_test

        test_score = mean_squared_error(y_pred=preds, y_true=y_true)
        test_score_res.append(test_score)
        clf_res.append(clf)
        print('[%3d/%4d] '%(i+1, 5), 'Test score: %.3f'%test_score)

    print("Test MSE average: %3f"%np.mean(test_score_res))

    return clf_res 

In [264]:
def feature(clf_res):
    importance_res = []
    for clf in clf_res:
        importance_clf =clf.feature_importances_
        importance_res.append(importance_clf)
    
    importance=[importance_res[0][i]/5+importance_res[1][i]/5+ importance_res[2][i]/5+ importance_res[3][i]/5+ importance_res[4][i]/5 for i in range(len(importance_res[1]))]
   
    feat_name_sort = data[features].columns
    important_features = pd.DataFrame([importance] , columns = feat_name_sort, index=['Importance'])     
    important_features =important_features.transpose().sort_values(by=['Importance'], ascending=False)
    return important_features

In [280]:
# Deprecated : best model is automatically loaded at end of fit
# clf.load_best_model()

def results(clf_res):
    preds_res =[]
    test_score_res = []
    for clf in clf_res:
        preds = clf.predict(X_test)



        y_true = y_test

        test_score = mean_squared_error(y_pred=preds, y_true=y_true)
        test_score_res.append(test_score)
        preds_res.append(preds)
     
    print(f"FINAL MSE SCORE  : {np.mean(test_score_res)}")
    test_score_final = np.mean(test_score_res)
    pred_final=[preds_res[0][i]/5+preds_res[1][i]/5+ preds_res[2][i]/5+ preds_res[3][i]/5+ preds_res[4][i]/5 for i in range(len(preds_res[1]))]
    #pred_final = pred_final/5
    return test_score_final, pred_final

# Training

In [225]:
class main():
    def __init__(self, data, y_test, X_test):
        max_hy = find_bestpar(data)
        clf_res = bestpar_tuning(data,max_hy)
        importance_feature = feature(clf_res)
        test_score_final, pred_final = results(clf_res)
        
        self.clf_res = clf_res
        self.pred_final = pred_final
        self.max_hy = max_hy
        self.test_score_final = test_score_final 
        self.X_test = X_test
        self.y_test = y_test
        self.importance_feature =  importance_feature


In [226]:
all_results=main(data, y_test, X_test)


Early stopping occurred at epoch 30 with best_epoch = 10 and best_valid_mse = 99.56798
Best weights from best epoch are automatically used!
[  1/   5]  Test score: 101.424

Early stopping occurred at epoch 89 with best_epoch = 69 and best_valid_mse = 87.30173
Best weights from best epoch are automatically used!
[  2/   5]  Test score: 86.955

Early stopping occurred at epoch 39 with best_epoch = 19 and best_valid_mse = 96.70667
Best weights from best epoch are automatically used!
[  3/   5]  Test score: 100.864

Early stopping occurred at epoch 25 with best_epoch = 5 and best_valid_mse = 102.83729
Best weights from best epoch are automatically used!
[  4/   5]  Test score: 101.991

Early stopping occurred at epoch 31 with best_epoch = 11 and best_valid_mse = 102.20493
Best weights from best epoch are automatically used!
[  5/   5]  Test score: 106.030
Test score average: 99.452696

Early stopping occurred at epoch 30 with best_epoch = 10 and best_valid_mse = 99.75107
Best weights from

Stop training because you reached max_epochs = 200 with best_epoch = 165 and best_valid_mse = 77.27714
Best weights from best epoch are automatically used!
[  1/   5]  Test score: 68.872

Early stopping occurred at epoch 195 with best_epoch = 145 and best_valid_mse = 82.60588
Best weights from best epoch are automatically used!
[  2/   5]  Test score: 71.113

Early stopping occurred at epoch 167 with best_epoch = 117 and best_valid_mse = 80.43525
Best weights from best epoch are automatically used!
[  3/   5]  Test score: 72.158

Early stopping occurred at epoch 165 with best_epoch = 115 and best_valid_mse = 82.60295
Best weights from best epoch are automatically used!
[  4/   5]  Test score: 69.137

Early stopping occurred at epoch 171 with best_epoch = 121 and best_valid_mse = 79.87124
Best weights from best epoch are automatically used!
[  5/   5]  Test score: 66.991

Test score average: 69.654178
FINAL TEST SCORE  : 69.65417815539116


# contribution of Importance

VENUE_LAT, USER_LAT, ITEM_COUNT are the top 3 important features to predict delivery time

In [265]:
important_feature = feature(clf_res)
important_feature

,Importance
VENUE_LAT,0.319221
USER_LAT,0.206502
ITEM_COUNT,0.127798
VENUE_LONG,0.112190
Weekday,0.060867
WIND_SPEED,0.058786
CLOUD_COVERAGE,0.037984
TEMPERATURE,0.023842
Hour_24,0.021264
PRECIPITATION,0.016977


### MSE from ESTIMATED_DELIVERY_MINUTES and ACTUAL_DELIVERY_MINUTES

TabNetregressor method's prediction is better than ESTIMATED_DELIVERY_MINUTES

In [281]:
test_score_final, pred_final = results(all_results.clf_res)

FINAL MSE SCORE  : 69.65417815539116


In [279]:
estimated = data[data.kfold == 5]['ESTIMATED_DELIVERY_MINUTES'].values.reshape(-1, 1)
print(f"ESTIMATED mse SCORE : {mean_squared_error(y_pred=estimated, y_true=y_test)}")


ESTIMATED mse SCORE : 81.46192733017378


# Save model and load

In [227]:
import pickle
with open("all_results.txt", "wb") as file: 
    pickle.dump(all_results, file)

# Intersting result
    if we include ACTUAL_DELIVERY_MINUTES - ESTIMATED_DELIVERY_MINUTES in the feature, MSE get lower

In [283]:
data_new = pd.read_csv('orders_autumn_2020 (1).csv')

unused_feat = ['Set', 'ESTIMATED_DELIVERY_MINUTES', 'AM_PM', 'TIMESTAMP']
categorical_columns = ['PRECIPITATION','CLOUD_COVERAGE','ITEM_COUNT' , 'Weekday' ,'Hour_24', 'Min']
features = [ col for col in data.columns if col not in unused_feat+[target]] 

cat_idxs = [ i for i, f in enumerate(features) if f in categorical_columns]



In [284]:
data_new=data_new.dropna().reset_index(drop=True)


In [285]:
target = 'ACTUAL_DELIVERY_MINUTES'
if "Set" not in data_new.columns:
    print("split data")
    data_new["Set"] = np.random.choice(["train", "valid", "test"], p =[.8, .1, .1], size=(data_new.shape[0],))


split data


In [286]:
X_train = data[features].values[train_indices]
y_train = data[target].values[train_indices].reshape(-1, 1)

X_valid = data[features].values[valid_indices]
y_valid = data[target].values[valid_indices].reshape(-1, 1)

X_test = data[features].values[test_indices]
y_test = data[target].values[test_indices].reshape(-1, 1)

clf = TabNetRegressor(cat_idxs=cat_idxs, verbose =0)
clf.fit(
    X_train=X_train, y_train=y_train,
    eval_set=[(X_train, y_train), (X_valid, y_valid)],
    eval_name=['train', 'valid'],
    eval_metric=['rmsle', 'mae', 'rmse', 'mse'],
    max_epochs=max_epochs,
    patience=50,
    batch_size=1024, virtual_batch_size=128,
    num_workers=0,
    drop_last=False
) 

Device used : cuda
epoch 0  | loss: 873.1683| train_rmsle: 4.79034 | train_mae: 252.3772| train_rmse: 270.59639| train_mse: 73222.40528| valid_rmsle: 4.74069 | valid_mae: 252.06542| valid_rmse: 270.69088| valid_mse: 73273.55145|  0:00:01s
epoch 1  | loss: 380.8524| train_rmsle: 6.22526 | train_mae: 357.30426| train_rmse: 381.72679| train_mse: 145715.34354| valid_rmsle: 6.22472 | valid_mae: 360.49453| valid_rmse: 386.09619| valid_mse: 149070.26865|  0:00:02s
epoch 2  | loss: 113.85715| train_rmsle: 2.40421 | train_mae: 110.83794| train_rmse: 130.61225| train_mse: 17059.5601| valid_rmsle: 2.29421 | valid_mae: 106.55007| valid_rmse: 127.26224| valid_mse: 16195.67818|  0:00:04s
epoch 3  | loss: 74.48745| train_rmsle: 0.26564 | train_mae: 16.534  | train_rmse: 39.2139 | train_mse: 1537.72966| valid_rmsle: 0.25006 | valid_mae: 16.05311| valid_rmse: 38.30812| valid_mse: 1467.51175|  0:00:05s
epoch 4  | loss: 55.06098| train_rmsle: 0.94647 | train_mae: 53.55697| train_rmse: 88.16435| train_mse

In [290]:
preds = clf.predict(X_test)

y_true = y_test

test_score = mean_squared_error(y_pred=preds, y_true=y_true)

print(f"BEST VALID SCORE   : {clf.best_cost}")
print(f"FINAL MSE SCORE   : {mean_squared_error(y_pred=preds, y_true=y_true)}")


BEST VALID SCORE   : 28.06004464641502
FINAL MSE SCORE   : 28.89202459497676


### Data argumntation using CTGAN


In [289]:
from ctgan import CTGANSynthesizer
data_for_argu= data.drop(['TIMESTAMP','kfold', 'ACTUAL_DELIVERY_MINUTES - ESTIMATED_DELIVERY_MINUTES', 'Hour','AM_PM'], axis = 1)
ctgan = CTGANSynthesizer()
ctgan.fit(data_for_argu,discrete_columns = categorical_columns, epochs =1000)
new_data = ctgan.sample(len(data_for_argu))

/home/ubuntu/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:70: FutureWarning: Pass n_components=10 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  FutureWarning)
/home/ubuntu/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:70: FutureWarning: Pass n_components=10 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  FutureWarning)
/home/ubuntu/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:70: FutureWarning: Pass n_components=10 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  FutureWarning)
/home/ubuntu/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:70: FutureWarning: Pass n_components=10 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  FutureWarning)
/home/ubuntu/anaconda3/lib/python3.6/site-packages/sklearn/utils/val

Epoch 1, Loss G: 2.3779, Loss D: -0.2650
Epoch 2, Loss G: 2.3674, Loss D: -0.0491
Epoch 3, Loss G: 1.6114, Loss D: 0.1248
Epoch 4, Loss G: 1.4835, Loss D: 0.1653
Epoch 5, Loss G: 1.1124, Loss D: 0.0819
Epoch 6, Loss G: 1.3637, Loss D: -0.2361
Epoch 7, Loss G: 0.7987, Loss D: 0.0827
Epoch 8, Loss G: 0.4993, Loss D: 0.1343
Epoch 9, Loss G: 0.0622, Loss D: 0.0604
Epoch 10, Loss G: -0.0815, Loss D: 0.0420
Epoch 11, Loss G: -0.5186, Loss D: -0.0532
Epoch 12, Loss G: -0.5687, Loss D: 0.0053
Epoch 13, Loss G: -0.7310, Loss D: -0.1926
Epoch 14, Loss G: -1.1745, Loss D: 0.1979
Epoch 15, Loss G: -1.0854, Loss D: -0.1483
Epoch 16, Loss G: -1.1642, Loss D: -0.0250
Epoch 17, Loss G: -1.3539, Loss D: 0.0405
Epoch 18, Loss G: -1.5691, Loss D: 0.0615
Epoch 19, Loss G: -1.6480, Loss D: 0.1665
Epoch 20, Loss G: -1.6246, Loss D: 0.0970
Epoch 21, Loss G: -1.4239, Loss D: -0.0457
Epoch 22, Loss G: -1.6344, Loss D: -0.0208
Epoch 23, Loss G: -1.6917, Loss D: -0.1057
Epoch 24, Loss G: -1.7456, Loss D: 0.0062


In [335]:
new_data

,Min,Hour_24,Weekday,ITEM_COUNT,USER_LAT,USER_LONG,VENUE_LAT,VENUE_LONG,ESTIMATED_DELIVERY_MINUTES,ACTUAL_DELIVERY_MINUTES,CLOUD_COVERAGE,TEMPERATURE,WIND_SPEED,PRECIPITATION,Gen
0,6,5,7,1,60.170810,24.922469,60.163098,24.932103,35,28,0.0,11.626737,5.542743,0.00000,1
1,11,18,7,7,60.186640,24.953406,60.182268,24.946714,42,40,0.0,18.001826,1.855721,0.00000,1
2,31,18,3,2,60.189162,24.939135,60.192409,24.923146,41,50,0.0,13.462145,5.612648,0.00000,1
3,47,10,1,1,60.174105,24.948457,60.193318,24.950790,40,37,0.0,13.469930,5.496389,6.31579,1
4,46,7,7,9,60.160081,24.931365,60.177639,24.931820,51,35,45.0,19.769396,5.857966,0.00000,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18424,16,24,6,1,60.199087,24.958764,60.184368,24.942788,32,31,0.0,16.188422,2.059208,0.00000,1
18425,35,15,7,6,60.159307,24.919529,60.155592,24.933509,39,42,45.0,16.132826,3.449769,0.00000,1
18426,37,15,6,2,60.154441,24.942143,60.166302,24.949844,29,33,0.0,12.791010,3.899196,0.00000,1
18427,37,17,2,6,60.187585,24.942744,60.180445,24.926411,24,37,0.0,21.788347,2.165013,0.00000,1


In [324]:
test_data = data_new[data_new.Set == 'test']  
train_data = data_new[data_new.Set != 'test']  

data_generated = new_data

# Add additional marker for visualization denoting generated data
data_generated ['Gen'] = 1

# Prepare augmented data which merges original + generated data
data_augmented = test_data.copy()
data_augmented['Gen']=0
data_augmented = data_augmented.append(data_generated )

In [325]:
unused_feat = ['Set', 'ESTIMATED_DELIVERY_MINUTES', 'ACTUAL_DELIVERY_MINUTES - ESTIMATED_DELIVERY_MINUTES', 'AM_PM', 'TIMESTAMP','kfold', 'Hour']
categorical_columns = ['PRECIPITATION','CLOUD_COVERAGE','ITEM_COUNT' , 'Weekday' ,'Hour_24', 'Min']
features = [ col for col in data.columns if col not in unused_feat+[target]] 

cat_idxs = [ i for i, f in enumerate(features) if f in categorical_columns]



In [326]:
data_augmented= data_augmented[features]

In [327]:
data_augmented

,Min,Hour_24,Weekday,ITEM_COUNT,USER_LAT,USER_LONG,VENUE_LAT,VENUE_LONG,CLOUD_COVERAGE,TEMPERATURE,WIND_SPEED,PRECIPITATION
1,17,6,6,8,60.163000,24.927000,60.153000,24.910000,0.0,15.000000,3.536440,0.00000
14,20,8,6,2,60.183000,24.923000,60.180000,24.927000,25.0,17.800000,3.350880,0.00000
16,25,8,6,1,60.183000,24.963000,60.194000,24.956000,25.0,17.800000,3.350880,0.00000
22,42,8,6,4,60.181000,24.954000,60.178000,24.949000,25.0,17.800000,3.350880,0.00000
64,48,10,6,2,60.195000,24.976000,60.185000,24.979000,0.0,17.800000,3.151930,0.00000
...,...,...,...,...,...,...,...,...,...,...,...,...
18424,16,24,6,1,60.199087,24.958764,60.184368,24.942788,0.0,16.188422,2.059208,0.00000
18425,35,15,7,6,60.159307,24.919529,60.155592,24.933509,45.0,16.132826,3.449769,0.00000
18426,37,15,6,2,60.154441,24.942143,60.166302,24.949844,0.0,12.791010,3.899196,0.00000
18427,37,17,2,6,60.187585,24.942744,60.180445,24.926411,0.0,21.788347,2.165013,0.00000


In [328]:
data_augmented=data_augmented.dropna().reset_index(drop=True)


In [329]:
target = 'ACTUAL_DELIVERY_MINUTES'
if "Set" not in data_augmented.columns:
    print("split data")
    data_augmented["Set"] = np.random.choice(["train", "valid", ], p =[.9, .1], size=(data_augmented.shape[0],))

train_indices = data_new[data_new.Set=="train"].index
valid_indices = data_new[data_new.Set=="valid"].index

X_train = data[features].values[train_indices]
y_train = data[target].values[train_indices].reshape(-1, 1)

X_valid = data[features].values[valid_indices]
y_valid = data[target].values[valid_indices].reshape(-1, 1)

X_test = test_data[features].values
y_test = test_data[target].values.reshape(-1, 1)



split data


In [331]:
clf = TabNetRegressor(cat_idxs=cat_idxs, verbose =1)
clf.fit(
    X_train=X_train, y_train=y_train,
    eval_set=[(X_train, y_train), (X_valid, y_valid)],
    eval_name=['train', 'valid'],
    eval_metric=['rmsle', 'mae', 'rmse', 'mse'],
    max_epochs=max_epochs,
    patience=50,
    batch_size=1024, virtual_batch_size=128,
    num_workers=0,
    drop_last=False
) 

Device used : cuda
epoch 0  | loss: 987.76137| train_rmsle: 8.52775 | train_mae: 549.29863| train_rmse: 549.47941| train_mse: 301927.61881| valid_rmsle: 8.4796  | valid_mae: 549.45452| valid_rmse: 549.63253| valid_mse: 302095.91489|  0:00:01s
epoch 1  | loss: 613.36462| train_rmsle: 4.14787 | train_mae: 205.59686| train_rmse: 205.99375| train_mse: 42433.42476| valid_rmsle: 4.11206 | valid_mae: 205.45267| valid_rmse: 205.85592| valid_mse: 42376.65785|  0:00:02s
epoch 2  | loss: 258.48559| train_rmsle: 0.52306 | train_mae: 27.30155| train_rmse: 33.04453| train_mse: 1091.94117| valid_rmsle: 0.52177 | valid_mae: 27.41577| valid_rmse: 33.53588| valid_mse: 1124.65515|  0:00:04s
epoch 3  | loss: 111.06707| train_rmsle: 0.299   | train_mae: 16.6701 | train_rmse: 21.38326| train_mse: 457.24402| valid_rmsle: 0.28697 | valid_mae: 16.30939| valid_rmse: 21.03215| valid_mse: 442.35116|  0:00:06s
epoch 4  | loss: 101.30225| train_rmsle: 0.30043 | train_mae: 16.76489| train_rmse: 19.32154| train_mse: 